# 네이버 카페 크롤러

네이버 카페 게시글을 검색 조건을 키워드로하여 크롤링 하는 코드 입니다.


[ 순서 ]
1. 라이브러리 import
2. 네이버 로그인
3. 검색 조건 지정하여 카페게시글 url 수집
4. url로 게시물 크롤링


## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import re #정규표현식
from selenium import webdriver #셀레니움 웹크롤러
from bs4 import BeautifulSoup as bs #웹크롤러
import requests #http 요청처리
import time
import warnings 
warnings.filterwarnings(action='ignore') #경고 메세지 안나오게 함

## 네이버 로그인

In [2]:
# 크롬 드라이버 제어
driver = webdriver.Chrome(r'C:\Tools\chromedriver_win32\chromedriver.exe')

# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
time.sleep(2) 

# 아이디& 비밀번호 입력
my_id = '' # "id" 대신에 자신의 네이버 아이디가 들어가야 합니다
my_pw = '' # "password" 대신에 자신의 네이버 비밀번호가 들어가야 합니다

# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# '로그인' 버튼 클릭
driver.find_element_by_id('log.login').click()
time.sleep(1)

## 검색 조건 지정하여 카페게시글 url 수집
        

(참고)
- 개별 게시판 내에 검색어 지정하지 않는 경우 페이지 제한없으나
- 조건 검색의 경우 최대 페이지 100개로 제한돼있음 (개별 게시판에 들어가서 검색어 사용하는 경우에도 100개 제한)

In [3]:
from selenium.webdriver.common.keys import Keys

post_url_list = [] # 조건에 맞는 게시물의 url 수집할 빈 리스트

# 네이버 카페 접속(ex. 나이키매니아)
# ***로그인한 네이버 계정이 해당 카페에 접근 권한이 있어야 함***
time.sleep(2)
cafe_url = 'https://cafe.naver.com/audiodudu' # 크롤링하기 원하는 카페 url 넣어주기
driver.get(cafe_url) #chromedriver가 위에 입력한 url 주소로 이동함
time.sleep(1)


keyword = input('키워드 입력하세요 : ') # 직접 input 받을 키워드, 해당 카페에서 수집하길 원하는 키워드를 입력함
search = driver.find_element_by_id('topLayerQueryInput') 
search.send_keys(keyword)
search.send_keys(Keys.RETURN)
time.sleep(1)


# 프레임 전환(네이버 카페는 여러 프레임으로 구성되어서 크롤링할 게시글이 있는 프레임으로 전환해주어야 함)
driver.switch_to.frame('cafe_main') 
time.sleep(1)


# 노출 게시물 개수 50개로 설정
number_of_post = driver.find_element_by_id('listSizeSelectDiv') #노출 게시물 개수 버튼 
number_of_post.click()
aa = number_of_post.find_elements_by_tag_name('li') #5개씩, 10개씩, 15개씩, 20개씩 중에
aa[len(aa)-1].click() #50개씩 선택


# 검색 세부 조건 지정 0) 제외단어 지정하기 1)게시판 지정  2)기간 설정  
# 0) 제외단어 지정하기

driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click() #상세검색 버튼 클릭
time.sleep(1)

bye = '판매, 팝니다' #조건 설정중 다음단어제외 조건 설정에 들어갈 단어 #판매글 제외하기 위해 사용
hi = '스피커' #조건 설정중 다음 단어중 1개이상 포함 조건 설정에 들어갈 단어 #나이키 의류에 한정하기 위해 사용

driver.find_element_by_xpath('//*[@id="srch_detail"]/div[2]/input').send_keys(bye) #다음단어제외 조건에 들어갈 단어 입력
driver.find_element_by_xpath('//*[@id="srch_detail"]/div[3]/input').send_keys(hi) #1개이상 포험 조건에 들어갈 단어 입력
time.sleep(1)


# 1) 게시판 지정
menu = driver.find_element_by_id('divSearchMenuTop') #해당 상세검색 창에서 전체 게시판 버튼을 눌러서
menu.click() 
menu_name = menu.find_element_by_class_name('select_list').text # 게시판 리스트를 가져옴
for i,j in enumerate(menu_name.split('\n')):
    print(i,j) # 게시판 종류 라벨과 함께 출력 
    
menu_number = [] #수집하길 원하는 게시판을 리스트에 저장하기 위해 빈리스트 생성
while True:        
    #수집하길 원하는 게시판을 리스트에서 찾아서 기입함
    i = input('크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :') 
    if i != '끝': # 게시판 번호 입력 다했으면 끝 이라고 쓰면 멈춥니다
        menu_number.append(int(i))
    else:
        break

aaa = menu.find_elements_by_tag_name('li') #게시판 선택하기 위해 게시판 리스트 element 전부 aaa에 담아둠 위 크롤링할 게시판 번호랑 연결시킬 것임
                                        #len(aaa) 하면 해당 카페의 전체 게시판 갯수만큼 나옴

for i in menu_number: #위에서 지정한 게시판 리스트들로 for문을 반복함
    aaa[i].click() #게시판 하나를 가져옴
    print()
    print('-----',menu_name.split('\n')[i],'-----') #어떤 게시판에서 데이터가 수집하고 있는지 출력. menu_name은 게시판 리스트 텍스트
    
    # 2) 기간 설정
    months = ['01','02','03', '04', '05', '06', '07', '08', '09', '11'] #검색 조건중 기간입력에 들어가는 월
    years = ['21'] #검색 조건중 기간입력에 들어가는 연도

    for year in years: #위에 지정한 연도에서 for문
        for month in months: #연도 안에서 월별로 for문
            
            driver.find_element_by_id('currentSearchDateTop').click() #기간입력 선택
            try:
                driver.find_element_by_id('input_1_top').click() #기간입력중 시작일자 지정
                driver.find_element_by_id('input_1_top').send_keys('20{0}{1}01'.format(year, month)) #시작일자
            except:
                break
                driver.implicitly_wait(30)
            try:
                driver.find_element_by_id('input_2_top').click() #기간입력중 종료일자 지정
                driver.find_element_by_id('input_2_top').send_keys('20{0}{1}30'.format(year, month)) #끝일자
            except:
                #종료일자가 0월 30일이 아니라 오늘 날짜가 돼야할 때 
                #끝일자를 오늘 날짜로 지정
                t = time.time()
                tmonth = int(time.strftime("%m", time.gmtime(t)))
                tdate = int(time.strftime("%d", time.gmtime(t)))-1
                driver.find_element_by_id('input_2_top').send_keys('20{}{}{}'.format(year,tmonth,tdate)) 
            driver.find_element_by_id('input_2_top').send_keys(Keys.RETURN) # 입력하고 엔터
            
            # 이제 연/월 하나의 쌍이 설정됨
            print('20{0}년 {1}월'.format(year, month)) #연/월 별로 데이터 수집되는 상황 출력
            
            driver.find_element_by_class_name('btn-search-green').click() # 조건 설정 완료 검색 버튼 클릭
            time.sleep(1)

            prev_next = driver.find_element_by_class_name('prev-next') # 아래 a가 들어있는 클래스
            buttons = prev_next.find_elements_by_tag_name('a') # 페이지 넘어가는 숫자 1~n

            page_num = 40 #수집하고 싶은 페이지 개수 지정 (게시물 개수 아님)
            
            if len(buttons) == 0: # 지정한 조건으로 게시물이 하나도 없을. 인식하는데 오래 걸림.
                pass
                
            else: #게시물이 있으면 url 수집함 
                try: # 중간중간 에러가 발생해서 전체를 try except안에 넣음
                    for p in range(1,page_num+1): #페이지마다 반복해서 수집 #1부터 page_num까지

                        # url 수집
                        soup = bs(driver.page_source, 'lxml') #p번째 페이지에 들어있는 게시글 50개(혹은 50개 미만)
                        post_url = soup.select('div .inner_list .article') #게시글 url이 inner_list > article에 들어있음        

                        # 카페 내 게시글의 url을 추출해서 만들어뒀던 post_url_list 리스트에 넣음
                        for i in post_url: #post_url엔 해당 페이지에 있는 게시글만큼 url이 들어있고
                            post_url_list.append('https://cafe.naver.com'+i['href']) #순서대로 저장함          
                        print(len(post_url_list)) #몇 개 수집됐는지 출력. #누적으로 나옴
                        driver.implicitly_wait(30)

                        prev_next = driver.find_element_by_class_name('prev-next') #아래 a가 들어있는 클래스
                        buttons = prev_next.find_elements_by_tag_name('a') # 페이지 넘어가는 숫자 1~n
                            
                            
                        # 페이지 이동 
                        # 페이지는 유형이 총 3가지가 있음
                        # 이전X 다음O 첫 페이지 / 이전O 다음O 중간 페이지 / 이전O 다음X 마지막 페이지
                        # 유형에 따라 buttons수가 달라짐
                        
                        if p<=10 & len(buttons)<12: #첫 페이지. #buttons의 수는 페이지수+다음버튼임 따라서 첫페이지라면 buttons수는 11(혹은 11미만)
                            if p%10 != 0: #10의 배수가 아니라면 
                                buttons[p].click() #p는 for문에서 1부터 시작하여 돌고있음. 여기서 button[0]은 1번째 페이지, button[1]은 2번째 페이지
                            else: #10의 배수라면 다음 버튼을 눌러줌
                                prev_next.find_element_by_css_selector('a.pgR').click()
                                
                        elif len(buttons) == 12: #이후 페이지 #다음 페이지라면 이전버튼+페이지수+다음버튼임 따라서 buttons수는 12
                            if p%10 != 0:
                                buttons[p%10+1].click() # p가 11이라면 button[2]를 누르게되는 것. 여기서 button[0]은 이전버튼, buttton[1]은 11페이지, button[2]는 12페이지 
                            else: 
                                prev_next.find_element_by_css_selector('a.pgR').click()

                        elif p>10 & len(buttons)<12 : #마지막 페이지 #이전버튼+페이지수임 따라서 buttons수는 11이하
                            buttons[p%10+1].click() 
                except: 
                    pass
                
    #해당 게시판에서 마지막 페이지까지 수집이 다 됐으면 다음 게시판으로 이동함
    menu = driver.find_element_by_id('divSearchMenuTop')
    menu.click() 
    aaa = menu.find_elements_by_tag_name('li')

키워드 입력하세요 : 스피커
0 전체 게시판
1 공지사항
2 가입인사는 이곳에 [등업]
3 자유 게시판[잡담환영]
4 나의 오디오 소개 (세팅/지름/자랑)
5 모임 후기 (정모, 청음회, 지역모임)
6 비청영상 모음
7 음악/음반/도서 소개
8 음악 플레이리스트 (강추)
9 안나의 이판살판
10 코난의 맛있는 오디오
11 김편의 음악싸롱
12 주간 재즈피플
13 공동구매 및 이벤트
14 7주년에디션 파워앰프 (두칠이)
15 두두오 포토이벤트 (이벤트용 임시 게시판)
16 대여이벤트 제품 수령신고
17 Q&A 궁금합니다.[기초질문 환영]
18 명소 그리고 추억
19 업체 홍보 (글 작성시 공지 숙지)
20 패시브 스피커 (사용기)
21 액티브 스피커 (사용기)
22 올인원 / 포터블 기기 (사용기)
23 DAC (사용기)
24 앰프 (사용기)
25 플레이어 (사용기)
26 케이블 (사용기)
27 액세서리 (사용기)
28 서브우퍼 (사용기)
29 헤드파이 제품 (사용기)
30 오디오 강좌, 제작, 아이디어
31 맥나의 룸 컨설팅
32 실명제 두두오 장터 (성실멤버 이상)
33 무료나눔 게시판
34 사건/사기/피해 정보
35 장터규칙위반(비실명,본인인증,양식)
36 운영진에게 바란다.[신문고]
37 운영진 게시판 (카페스탭)
38 가입회원 현황
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :4
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :17
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :20
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :21
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :22
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :23
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력  :24
크롤링할 게시판 번호를 하나씩 입력하세요 ex) 8  다했으면 끝 입력

In [4]:
#수집한 url 확인
#set 함수로 중복 확인
#url은 안겹처도 전체 게시판과 다른 게시판에 중복되는 게시물이 있기 때문에 추후 중복 데이터 제거 필요
print(len(post_url_list), len(set(post_url_list)))
for i in post_url_list:
    print(i)

11471 10539
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=1&userDisplay=50&menuid=18&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-01-012021-01-30&media=0&sortBy=date&articleid=298799&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=1&userDisplay=50&menuid=18&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-01-012021-01-30&media=0&sortBy=date&articleid=298793&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=1&userDisplay=50&menuid=18&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A

https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=5&userDisplay=50&menuid=18&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-08-012021-08-30&media=0&sortBy=date&articleid=374833&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=5&userDisplay=50&menuid=18&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-08-012021-08-30&media=0&sortBy=date&articleid=374798&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=5&userDisplay=50&menuid=18&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&sea

https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=20&userDisplay=50&menuid=5&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-01-012021-01-30&media=0&sortBy=date&articleid=285288&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=20&userDisplay=50&menuid=5&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-01-012021-01-30&media=0&sortBy=date&articleid=285283&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=20&userDisplay=50&menuid=5&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&sea

https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=5&userDisplay=50&menuid=5&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-03-012021-03-30&media=0&sortBy=date&articleid=321173&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=5&userDisplay=50&menuid=5&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&searchdate=2021-03-012021-03-30&media=0&sortBy=date&articleid=321164&referrerAllArticles=false
https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=28248719&page=5&userDisplay=50&menuid=5&inCafeSearch=true&searchBy=0&query=%EC%8A%A4%ED%94%BC%EC%BB%A4&includeAll=&exclude=%ED%8C%90%EB%A7%A4%2C+%ED%8C%9D%EB%8B%88%EB%8B%A4&include=%EC%8A%A4%ED%94%BC%EC%BB%A4&exact=&search

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
#수집한 url dataframe으로 저장
nm_url = pd.DataFrame(post_url_list) #리스트로 dataframe 만들기
#nm_url.to_csv('1_nm_url_list.csv') #dataframe csv 형태로 저장하기 
nm_url.to_csv('1_nm_url_list_4team.csv') #dataframe csv 형태로 저장하기 
nm_url #만든 dataframe 출력해서 확인해보기

NameError: name 'post_url_list' is not defined

## url로 게시물 크롤링

- 위에 1.0.1 라이브러리 import와 1.0.2 네이버 로그인이 실행된 상태에서 진행해야함 

In [4]:
#post_url_list = pd.read_csv('1_nm_url_list.csv') #위에서 저장 url 파일 가져오기
post_url_list = pd.read_csv('1_nm_url_list_4team.csv')
a = post_url_list['0']

In [5]:
a 

0        https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
1        https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
2        https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
3        https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
4        https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
                               ...                        
11466    https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
11467    https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
11468    https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
11469    https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
11470    https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...
Name: 0, Length: 11471, dtype: object

In [6]:
len(a)

11471

In [22]:
# 제목, 본문, 댓글, 날짜 빈 list 생성
titles = []
reviews = []
comments = []
time_list = []

slicing_num = 2400
# 수집한 url 이 담겨있는 a에서 for문 반복
for url in a[1600:slicing_num]: #샘플로 슬라이싱 10으로 해둠**
        
    driver.get(url) #해당 url로 이동
    driver.implicitly_wait(30)

    # 프레임 전환 (여기까진 동적크롤러)
    try: 
        driver.switch_to.frame('cafe_main') 
        time.sleep(0.8)
        soup = bs(driver.page_source, 'lxml') 
    
    except UnexpectedAlertPresentException as e:
        pass
        
    
    # 데이터 수집은 정적크롤러
    # 제목 수집
    try:
        title = soup.find('h3',class_='title_text').text.strip() #제목 찾아서 수집
        titles.append(title)
    except: 
        try:
            title = driver.find_element_by_css_selector('h3.title_text').text.strip() #위 코드가 작동 안하는 경우가 있어서 같은 동작 다른 코드
            titles.append(title)
            time.sleep(0.8)
        except:
            title = [] #위 두개 코드로도 작동이 안되면 빈 리스트로 두고 다음 게시물로 넘어가도록 함
            titles.append(title)   
        
    # 본문 수집
    try:
        content = soup.select_one('div.se-module.se-module-text').text.strip() #본문 찾아서 수집
        reviews.append(content)
    except :
        try:
            content = soup.select_one('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우가 있어서 다른 코드
            reviews.append(content)
        except :
            try: 
                content = driver.find_element_by_css_selector('div.se-module.se-module-text').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                reviews.append(content)
            except :
                try:
                    content = driver.find_element_by_css_selector('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                    reviews.append(content)
                except :
                    content = [] #빈 리스트로 두고 다음 게시물로 넘어가도록 함
                    reviews.append(content)
                     
    # 댓글 수집
    iscomment = soup.find_all('span',class_='text_comment')
    if len(iscomment) != 0: # 댓글이 0개가 아니면 
        box = []
        for i in iscomment:
            box.append([i.get_text()])
    else: # 댓글이 0개면 넘어감
        box = []
        pass
    comments.append(box)
    
    #날짜 수집
    try:
        timeline = driver.find_element_by_xpath("/html/body/div/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]").text.strip()
    except:
        timeline = driver.find_element_by_css_selector("#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.date").text.strip()  
    time_list.append(timeline)

In [23]:
#수집된 제목, 본문, 댓글 확인 
#3개 숫자가 맞아야 dataframe 만들 수 있음
print(len(titles))
print(len(reviews))
print(len(comments))
print(len(time_list))

#오류로 인해 안 맞으면 아래 코드로 맨뒤 게시글 하나 삭제해서 갯수 맞춰줌
#titles = titles[:-1] #타이틀이 본문이나 댓글보다 하나 많으면 슬라이싱 해줌
#len(titles)

800
800
800
800


In [24]:
#Dataframe 형태로 위에서 수집한 제목, 본문, 댓글을 합쳐줌
data = {'tit' : titles,
        'body' : reviews,
        'comment' : comments,
        'time' : time_list}

dataDF = pd.DataFrame(data)

In [25]:
#csv로 저장
#dataDF['source'] = 'A' #해당 데이터 표시하기 위해 필요한 정보 추가 할 수 있음. #어떤 카페에서 수집했는지
dataDF['keyword'] = '스피커' #어떤 키워드로 데이터를 수집했는지 추가
dataDF.to_csv("1_nm_data_" + str(slicing_num) + ".csv", encoding = "utf-8-sig") #한글 인코딩 utf-8-sig 로 하면 안깨짐
dataDF #저장된 모습 확인

,tit,body,comment,time,keyword
0,셋팅 조언좀 얻고싶은 초보입니다,여기서 좀더 좋은셋팅이있을까하고 자문을 구하는 오린입니다,[[눈대중과 감으로 한셋팅입니다 오린이입니다 평소 소리는 핸드폰어플 90정도듣습니다...,2021.08.13. 20:15,스피커
1,제오디오 시스템 입니다,[],"[[딱 보는순간 어디서 구입하신건지 다 알아버렸습니다ㅋㅋ], [^-^;;대단하시네요...",2021.08.13. 12:25,스피커
2,B&W 705 s2 들였습니다,사무실 책상에서 사용중인 jbl a130 스피커가 디자인이 마음에 들고 소리도 좋지...,"[[ceo ^^], [감사합니다 ^^], [멋진 스피커 영입 축하드립니다. 스파이크...",2021.08.13. 12:18,스피커
3,간만에 지름신고 합니다.,안녕하세요~집나갔던(?) 선달이 오랜만에 친정에 문안 인사겸 지름신고 합니다~~​3...,"[[], [감사합니다 ~~^^], [엄청 실하게 생겼습니다~\n좋은 케이블 들이신것...",2021.08.13. 07:12,스피커
4,정들었던 탄호이저와의 이별신고,안녕하세요.. 그동안 눈팅으로 도움은 엄청 받았으나 도움 드리는 게시글이 없어...,"[[], [읽다 보니 괜히 슬퍼지네요^^], [탄호이저가 소리더좋을텐데요..], [...",2021.08.13. 00:59,스피커
...,...,...,...,...,...
795,"턴테이블 on/off시 스피커에서 퍽~소리가 납니다. mm은 미미한 퍽소리인데,m...",제목이 내용이 되어버렸네요.그래서 on/off시 볼륨을 줄인후껐다 켰다를 하면서 L...,"[[혹시 스피커에는 문제가 없을까요?], [포노앰프 킨 상태에서 끄시나요?], [인...",2021.01.20. 02:24,스피커
796,제네렉 스피커 선택?,현재 제네렉8030c 로 음감중입니다. 듣다보니 제네렉이 맘에들어 업글을 생각중인데...,[[저는 현재 사방 이미터 정도돼는곳에서8020d 를 사용중에 있습니다제생각에는83...,2021.01.20. 01:57,스피커
797,마란츠 PM7005 와 스피커 조합,안녕하세요~선물 받은 마란츠 PM7005 앰프가 있는데 스피커를 어떤걸 사용하면 좋...,[[비앤더블류 톨보이 추천합니다. 34평이라니 북셀프는 감당이 좀. 스피커를 맘에 ...,2021.01.19. 23:51,스피커
798,어댑터 비교 질문 : 19.5v 7.7a vs 24v 5a,"초짜라 궁금해서 여쭤봅니다.​알리발 미니앰프를 구매했는데요, 어댑터 미포함 제품입니...",[[총 W도 중요하지만보통 V가 높아야 구동력이 더 좋습니다!!!19v해보시고 스피...,2021.01.19. 23:10,스피커


In [7]:
start_num = 1600
end_num = 1600 + 800

while True:
    start_num = end_num
    end_num = end_num + 800
    if end_num > len(a) -1 :
        break
        
    # 제목, 본문, 댓글, 날짜 빈 list 생성
    titles = []
    reviews = []
    comments = []
    time_list = []

    #slicing_num = 2400
    # 수집한 url 이 담겨있는 a에서 for문 반복
    for url in a[start_num:end_num]: #샘플로 슬라이싱 10으로 해둠**

        driver.get(url) #해당 url로 이동
        driver.implicitly_wait(30)

        # 프레임 전환 (여기까진 동적크롤러)
        try: 
            driver.switch_to.frame('cafe_main') 
            time.sleep(0.8)
            soup = bs(driver.page_source, 'lxml') 

        except UnexpectedAlertPresentException as e:
            pass


        # 데이터 수집은 정적크롤러
        # 제목 수집
        try:
            title = soup.find('h3',class_='title_text').text.strip() #제목 찾아서 수집
            titles.append(title)
        except: 
            try:
                title = driver.find_element_by_css_selector('h3.title_text').text.strip() #위 코드가 작동 안하는 경우가 있어서 같은 동작 다른 코드
                titles.append(title)
                time.sleep(0.8)
            except:
                title = [] #위 두개 코드로도 작동이 안되면 빈 리스트로 두고 다음 게시물로 넘어가도록 함
                titles.append(title)   

        # 본문 수집
        try:
            content = soup.select_one('div.se-module.se-module-text').text.strip() #본문 찾아서 수집
            reviews.append(content)
        except :
            try:
                content = soup.select_one('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우가 있어서 다른 코드
                reviews.append(content)
            except :
                try: 
                    content = driver.find_element_by_css_selector('div.se-module.se-module-text').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                    reviews.append(content)
                except :
                    try:
                        content = driver.find_element_by_css_selector('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                        reviews.append(content)
                    except :
                        content = [] #빈 리스트로 두고 다음 게시물로 넘어가도록 함
                        reviews.append(content)

        # 댓글 수집
        iscomment = soup.find_all('span',class_='text_comment')
        if len(iscomment) != 0: # 댓글이 0개가 아니면 
            box = []
            for i in iscomment:
                box.append([i.get_text()])
        else: # 댓글이 0개면 넘어감
            box = []
            pass
        comments.append(box)

        #날짜 수집
        try:
            timeline = driver.find_element_by_xpath("/html/body/div/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]").text.strip()
        except:
            timeline = driver.find_element_by_css_selector("#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.date").text.strip()  
        time_list.append(timeline)


    print(len(titles))
    print(len(reviews))
    print(len(comments))
    print(len(time_list))


    data = {'tit' : titles,
        'body' : reviews,
        'comment' : comments,
        'time' : time_list}

    dataDF = pd.DataFrame(data)

    dataDF['keyword'] = '스피커' #어떤 키워드로 데이터를 수집했는지 추가
    dataDF.to_csv("1_nm_data_" + str(end_num) + ".csv", encoding = "utf-8-sig") #한글 인코딩 utf-8-sig 로 하면 안깨짐
    dataDF

800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800
800


In [4]:
import os
import pandas as pd

naver_cafe_all_data = []
naver200 = pd.read_csv('1_nm_data_200.csv')
naver_cafe_all_data.append(naver200)
#naver_cafe_all_data

In [5]:

merge_idx = 800
for i in range(14):    
    naver_more_data = pd.read_csv('1_nm_data_' + str(merge_idx) + '.csv')
    naver_cafe_all_data.append(naver_more_data)
    print('1_nm_data_' + str(merge_idx) + '.csv', 'has been merged')
    merge_idx += 800
    
naver_speaker_data = pd.concat(naver_cafe_all_data, axis=0, ignore_index=True)

1_nm_data_800.csv has been merged
1_nm_data_1600.csv has been merged
1_nm_data_2400.csv has been merged
1_nm_data_3200.csv has been merged
1_nm_data_4000.csv has been merged
1_nm_data_4800.csv has been merged
1_nm_data_5600.csv has been merged
1_nm_data_6400.csv has been merged
1_nm_data_7200.csv has been merged
1_nm_data_8000.csv has been merged
1_nm_data_8800.csv has been merged
1_nm_data_9600.csv has been merged
1_nm_data_10400.csv has been merged
1_nm_data_11200.csv has been merged


In [6]:
naver_speaker_data

,Unnamed: 0,tit,body,comment,time,keyword
0,0,탄노이,예전에 마란츠와JBL이 세상에서 제일 좋은 전축이라 알던시절에 가슴 깊은곳 부터 조...,"[['아덴 축하 드립니다 ^^'], ['감사 합니다.'], ['득탬과 동시에 지름 ...",2021.01.30. 23:30,스피커
1,1,새로운 친구를 들였습니다. (feat 다인오디오 이보크20 Vs B&W 702S2),안녕하세요rose 150 유니코듀에 B&W 702S2로 만족하면서 듣고 있던중여러...,"[['마음을 울리는 소리 저도 느끼고 싶어요..'], ['다인오디오 한번 들어보실만...",2021.01.30. 23:03,스피커
2,2,올해 음반은 이제 멈추려합니다.,드보르작 연주가 좋아서 부페오와 이반 피셔의 연주반을 모았네요. 이제 음반 헌팅은 ...,"[['음반욕심은 이상하게 끊어지지를 않네요~^^'], ['그니깐요^~^'], ['...",2021.01.30. 20:30,스피커
3,3,Bryston 4B-sst2 파워앰프 + Schiit Freya plus 구입했습니다.,잘지내셨나요? ㅎ한동안 음악만 듣고 뽐뿌는 안받을려고 6개월 정도는 두우오를 외면 ...,"[['멋지네요'], ['감사합니다. 미천합니다'], ['멋집니다..틸에 브라이스턴이...",2021.01.30. 18:20,스피커
4,4,오린이가 피에가 프리미엄 501 질렀어요.,두두오에 가입한 지 얼마 안 되는 오린입니다. 아침밥 먹고 쉬고 있는데 택배가 왔더...,"[[''], ['감사합니다.'], ['외관이 멋지면 당연히 소리도 좋겠지요. 축하드...",2021.01.30. 18:19,스피커
...,...,...,...,...,...,...
11395,795,[런던오디오] 언바란스 인터케이블(Hybrid-R) 체험이벤트 후기,https://cafe.naver.com/common/storyphoto/viewe...,"[['정성어린 상세한 후기에 진심으로 감사드립니다.^^'], ['후기 잘 봤습니다~...",2021.01.16. 23:46,스피커
11396,796,문의) 에코세스피커케이블 vs. 오디오퀘스트케이블 문의,안녕하세요?에코세 MS2.15 스피커케이블과 오디오퀘스트 로켓44 케이블이 유사한 ...,[],2021.01.13. 10:31,스피커
11397,797,바이와이어링을 하니까 파바로티의 목소리가 고역에서 금속성의 소리가 덜 나옵니다.,위의 사진은 네오텍 NS-1410 스피커 케이블의 사진입니다. 4일 전에 샀습니다....,[['중역이 좋아 진것 같네요올라가지도 않는 노래를억지로 부르는 노래방 소리에서해방...,2021.01.11. 17:47,스피커
11398,798,소름이 쫙쫙 돋습니다 ㅎㅎㅎㅎㅎ,벤션 CAT.8 (1.5미터) 소리가 뚫렸습니다^___^ )))역시 최소 3일은 지...,[['올만에~~~~~~~^^\n애정하는 선재(케이블)당원님♥♥♥\n축하합니당~ㅎㅎㅎ...,2021.01.11. 15:26,스피커


In [7]:
naver_speaker_data.to_csv('naver_cafe_speaker_all_data.csv',index=False,encoding='utf-8-sig')